In [1]:
import os

from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [2]:
spark = SparkSession.builder \
    .appName("pdf-retrieval") \
    .master("spark://spark-master:7077") \
    .config("spark.eventLog.enabled", "true") \
    .config("spark.eventLog.dir", "file:///opt/spark/spark-events/") \
    .config("spark.executor.memory", "4g") \
    .config("spark.executor.cores", 1) \
    .config("spark.dynamicAllocation.enabled", "true") \
    .config("spark.dynamicAllocation.shuffleTracking.enabled", "true") \
    .config("spark.dynamicAllocation.initialExecutors", 0) \
    .config("spark.dynamicAllocation.minExecutors", 1) \
    .config("spark.dynamicAllocation.maxExecutors", 10) \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .config("spark.scheduler.mode", "FAIR") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/28 13:38:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/28 13:38:29 WARN Utils: spark.dynamicAllocation.initialExecutors less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, please update your configs.
24/09/28 13:38:29 WARN Utils: spark.executor.instances less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, please update your configs.


In [31]:
df = spark.read.parquet("../hfdata/script_patched_2024")
df.show()

+---------------+--------------------+------------------+--------------------+--------------------+----+
|             ip|                host|            server|              emails|    script_src_attrs|year|
+---------------+--------------------+------------------+--------------------+--------------------+----+
|   217.160.0.83|https://nadinesim...|            Apache|mailto:nadine@nad...|https://nadinesim...|2024|
| 185.127.236.35|https://azionecat...|         Glaucorm3|                    |\'https://azionec...|2024|
|   52.15.227.12|https://secureftp...|              NULL|                    |js/dist/jquery.mi...|2024|
|   2.59.135.142|https://www.zeitk...|             nginx|                    |/_assets/a739cde7...|2024|
|    104.21.60.7|http://www.250r.r...|        cloudflare|                    |http://www.250r.r...|2024|
|  46.183.11.162|https://taliarand...|             nginx|mailto:hello@tali...|https://taliarand...|2024|
|    65.60.61.13|https://almost4x4...|            Apach

In [32]:
hf_df = df.filter(col('host').rlike(r'(huggingface.co)')).select('host')
hf_df.show(n=20, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------+
|host                                                                                                                       |
+---------------------------------------------------------------------------------------------------------------------------+
|https://huggingface.co/login?next=%2Fpapers%2F2307.10936                                                                   |
|https://huggingface.co/login?next=%2Fspaces%2Fabdulmatinomotoso%2Fstock_news_sentiment_and_NER%2Fdelete%2Fmain%2Fapp.py    |
|https://huggingface.co/tensorspeech/tts-mb_melgan-baker-ch/raw/main/.gitattributes                                         |
|https://huggingface.co/docs/transformers/v4.27.2/en/fast_tokenizers                                                        |
|https://huggingface.co/datasets/GEM-submissions/Simon1997__bart-base_original_cacapo__1678442469/discussions?status=o

In [33]:
hf_df.count()

235

In [34]:
openai_df = df.filter(col('host').rlike(r'(openai.com)')).select('host')
openai_df.show(n=20, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------+
|host                                                                                                                             |
+---------------------------------------------------------------------------------------------------------------------------------+
|https://community.openai.com/t/please-accommodate-desktop-users-with-a-wider-text-area/620797                                    |
|https://community.openai.com/posts.rss                                                                                           |
|https://help.openai.com/en/articles/8174106-how-to-login-to-the-android-app-with-microsoft-authentication                        |
|https://community.openai.com/t/gpt4-no-longer-provides-decent-tables/456686                                                      |
|https://openai.com/index/openai-scholars-2018-meet-our-scholars/           

In [35]:
openai_df.count()

42

In [27]:
pdf_df = df.filter(col('host').rlike(r'(?i).pdf')).select("host")
pdf_df.show(n=20, truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|host                                                                                                                                                                                                                                                                                                                                                              |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [24]:
pdf_df.count()

NameError: name 'pdf_df' is not defined

In [29]:
cloud_df = pdf_df.filter(col('host').rlike(r'(?i)\b(aws|amazon|case study)')).select("host")
cloud_df.show(n=5, truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|host                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
+-----------------------------------------------------------

In [30]:
cloud_df.count()

1374

In [31]:
arxiv_df = pdf_df.filter(col('host').rlike(r'(i?)(arxiv)'))
arxiv_df.show(n=25, truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------+
|host                                                                                                                                                         |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------+
|http://export.arxiv.org/pdf/math/0409192                                                                                                                     |
|http://export.arxiv.org/pdf/1410.6310                                                                                                                        |
|https://arxiv.org/pdf/1912.00998.pdf                                                                                                                         |
|http://export.arxiv.org/pdf/1410.3382  

In [32]:
arxiv_df.count()

144